# 1. Prepare data
- Read in driving data and surrounding vehicle information
- Read in driver id and subjective risk evaluators

In [1]:
# Read in information of driver, lane change, and risk labels

import os
import pandas as pd

DRIVER_IDS = ['m9600A', 'm9601A', 'm9602B',
             'm9603A', 'm9604A', 'f9605A',
             'm9606A', 'f9608A', 'm9609A', 'm9611A']
DATA_DIR = '/media/hona/AquaAge_1/lc_data/'
RISK_DIR = os.path.join(DATA_DIR, '03_risk_level_data')

label_csv = pd.read_csv(os.path.join(RISK_DIR, '9600_lane_change_risk_scores_classification.csv'))

COL_LANE_CHANGE_INFO = ['driver', 'task', 'trip_id', 'LC_type', 'start_frame', 'end_frame']
COL_RISK_LABELS = ['score_1', 'score_2', 'score_3', 'score_4', 'score_5', 'score_6',
               'score_7', 'score_8', 'score_9', 'score_10', 'normalized_risk',
               'objective_risk_label', 'S_P1', 'S_P2', 'S_P3', 'S_P4', 'S_P5', 'S_P6',
               'S_P7', 'S_P8', 'S_P9', 'S_P10']
LANE_CHANGE_INFO = label_csv[COL_LANE_CHANGE_INFO]
RISK_LABELS = label_csv[COL_RISK_LABELS]
print(LANE_CHANGE_INFO.head(5))

lane_change_cnt = len(LANE_CHANGE_INFO)
print(lane_change_cnt)


   driver  task  trip_id LC_type  start_frame  end_frame
0  f9605A     1     2014   right         6387       6462
1  f9605A     1     2014    left         7053       7132
2  f9605A     1     2014   right         5534       5601
3  f9605A     1     2014    left         5301       5364
4  f9605A     1     2014    left         1578       1649
988


In [3]:
# Read in surrounding vehicle information
def get_surrounding_data(lane_change_id):
    SURR_DIR = os.path.join(DATA_DIR, '06_surrounding_vehicle_info')
    driver_id = LANE_CHANGE_INFO['driver'][lane_change_id]
    task_id = LANE_CHANGE_INFO['task'][lane_change_id]
    lc_type = LANE_CHANGE_INFO['LC_type'][lane_change_id]
    start_frame = LANE_CHANGE_INFO['start_frame'][lane_change_id]
    end_frame = LANE_CHANGE_INFO['end_frame'][lane_change_id]
    
    for i in range(len(TIME_VARIANTS)):
        # Get time length
        t = TIME_VARIANTS[i]*10 # 10hz sampled
        start_frame = start_frame - t
        end_frame = end_frame + t
        
        if os.path.isdir(SURR_DIR):
            surr_file = SURR_DIR + '/' + driver_id + 'ex' \
                + str(task_id) + '-SurVehicleInfo.csv'
            if os.path.isfile(surr_file):
                surr_pd = pd.read_csv(surr_file)
                frame_max = len(surr_pd)    
                
                if start_frame < 0:
                    start_frame = 0  
                if end_frame >= frame_max:
                    end_frame = frame_max
                if lane_change_id % 100 == 0:
                    print('Loading surrounding vehicle information for lane change No.{}'.format(lane_change_id+1))
                each_surr_data = surr_pd.iloc[start_frame:end_frame,:]
                if lane_change_id == lane_change_cnt-1:
                    print('Surrounding vehicle information data is loaded.\n')    
            else:
                if driver_id == 'm9607A':
                    pass
                else:
                    print("File {} does not exsit.".format(surr_file))
        else:
            print("Folder {} does not exsit.".format(SURR_DIR))
        return each_surr_data
             

In [5]:
# Read in driving data
DRV_DIR = os.path.join(DATA_DIR, '01_driving_data')
#TIME_VARIAENTS = [1, 3, 5, 10] # Cut data before&after 1,3,5,10 seconds
TIME_VARIANTS = [3] # Cut data before&after 1,3,5,10 seconds

DATA = []
# Organize ego-vehicle pose and surrouding vehicle info in lane change order.
for lane_change_id in range(len(LANE_CHANGE_INFO)):
    driver_id = LANE_CHANGE_INFO['driver'][lane_change_id]
    task_id = LANE_CHANGE_INFO['task'][lane_change_id]
    lc_type = LANE_CHANGE_INFO['LC_type'][lane_change_id]
    start_frame = LANE_CHANGE_INFO['start_frame'][lane_change_id]
    end_frame = LANE_CHANGE_INFO['end_frame'][lane_change_id]
    
    for i in range(len(TIME_VARIANTS)):
        # Get time length
        t = TIME_VARIANTS[i]*10 # 10hz sampled
        start_frame = start_frame - t
        end_frame = end_frame + t
        
        each_drv_dir = os.path.join(DRV_DIR, driver_id)
        if os.path.isdir(each_drv_dir):
            drv_file = each_drv_dir +'/drv/ex' + str(task_id) +'/' \
                + driver_id + 'ex' + str(task_id) + '_drv.csv'
            if os.path.isfile(drv_file):
                drv_pd = pd.read_csv(drv_file)
                frame_max = len(drv_pd)    
                
                if start_frame < 0:
                    start_frame = 0  
                if end_frame >= frame_max:
                    end_frame = frame_max
                if lane_change_id % 100 == 0:
                    print('Loading driving data for lane change No.{}'.format(lane_change_id+1))    
                each_drv_data = drv_pd.iloc[start_frame:end_frame,:]
                each_surr_data = get_surrounding_data(lane_change_id)
                
                data = {'lane_change_id': lane_change_id + 1,
                    'lane_change_info': LANE_CHANGE_INFO.iloc[lane_change_id,:],
                     'risk_labels': RISK_LABELS.iloc[lane_change_id,:],
                     'drv_data': each_drv_data,
                     'surr_info': each_surr_data }
                DATA.append(data)
                if lane_change_id == lane_change_cnt-1:
                    print('\nData loaded.\n')
            else:
                print("File {} does not exsit.".format(drv_file))
        else:
            if driver_id == 'm9607A':
                pass
            else:
                print("Folder {} does not exsit.".format(each_drv_dir))

Loading driving data for lane change No.1
Loading surrounding vehicle information for lane change No.1
Loading driving data for lane change No.101
Loading surrounding vehicle information for lane change No.101
Loading driving data for lane change No.201
Loading surrounding vehicle information for lane change No.201
Loading driving data for lane change No.301
Loading surrounding vehicle information for lane change No.301
Loading driving data for lane change No.401
Loading surrounding vehicle information for lane change No.401
Loading driving data for lane change No.501
Loading surrounding vehicle information for lane change No.501
Loading driving data for lane change No.601
Loading surrounding vehicle information for lane change No.601
Loading driving data for lane change No.801
Loading surrounding vehicle information for lane change No.801
Loading driving data for lane change No.901
Loading surrounding vehicle information for lane change No.901
Surrounding vehicle information data is l

In [6]:
import pickle
pickle_name = 'pickle/Risk_DATA_' + str(TIME_VARIANTS[0]) + '.pkl'
with open(pickle_name, "wb") as f:
    pickle.dump(DATA, f) #保存

In [7]:
def dynamic(signal):
    N = 10
    o_t = []
    T = len(signal)
    for t in range(T):
        if t < (T - 2*N):
            sum1 = sum(n * signal.iloc[t + n] for n in range(2 * N) if t + n < T)
            sum2 = sum(n * n for n in range(2 * N) if t + n < T)
            if sum2 == 0:
                sum2 = 1
            else:
                o_t.append(sum1 / sum2)
        else:
            o_t.append(o_t[t-1])
    return o_t

In [8]:
def get_dynamic_data(x):
    """
    Argument:
    x --  driving signals
    
    Returns:
    X -- driving signals with first order and second order signals
    """
    feature_names = ['brake', 'accel',  'velocity', 'steering_angle', 'a_lon', 'a_lat']
    dynamic_features = ['brake', 'accel', 'steering_angle']
    X = pd.DataFrame(data=x)
    X.columns = feature_names
    
    for feature in feature_names:
        f1_feature = '1st_order_' + feature
        o_t = dynamic(X[feature])
        X[f1_feature] = o_t
        s2_feature =  '2nd_order_' + feature
        o_o_t = dynamic(X[f1_feature])
        X[s2_feature] = o_o_t
        
    return X

#### 衝突余裕時間 (TTC)

ttc = relative distance / relative_speed

国土交通省が定めた技術指針によると，衝突被害軽減ブレーキシステムなどにおいて自動ブレーキが作動するタイミングは，乗用車で TTC が 1.4 秒以下，大型車で 1.6 秒以下と定められている．

In [9]:
import numpy as np
def divide_surr_zone(surr_info):
    frame_num, _ = surr_info.shape
    vehicle_num = int(_/4)
    surr_zones_ttc = np.zeros((frame_num,10))
    #y:longidunal direction, x:lateral direction
    
    for i_frame in range(frame_num):
        for i in range(vehicle_num):
            
            if np.sum(surr_info.iloc[i_frame,i*4:i*4+4]) == 0:
                zone_id = 0 #vehicle has no data
            else:
                #'vehicle1-relx(m)', 'vehicle1-rely(m)', 'vehicle1-relvx(m/s)','vehicle1-relvy(m/s)'
                relx = surr_info.iloc[i_frame,i*4]
                rely = surr_info.iloc[i_frame,i*4+1]
                relvx = surr_info.iloc[i_frame,i*4+2]
                relvy = surr_info.iloc[i_frame,i*4+3]
                ttcx = relx / (relvx + 0.000001)
                ttcy = rely / (relvy + 0.000001)
                
                if rely < 50 and rely > 10 and relx > 1.625:
                    zone_id = 1
                elif rely < 10 and rely > -10 and relx > 1.625:
                    zone_id = 2
                elif rely < 50 and rely > 0 and relx < 1.625 and relx > -1.625:
                    zone_id = 3
                elif rely < 50 and rely > 10 and relx < -1.625:
                    zone_id = 4
                elif rely < 10 and rely > -10 and relx < -1.625:
                    zone_id = 5
                else:
                    zone_id = 0
            if zone_id != 0:
                surr_zones_ttc[i_frame][(zone_id-1)*2] = ttcx
                surr_zones_ttc[i_frame][(zone_id-1)*2+1] = ttcy
        
    cols = ['zone_1_x', 'zone_1_y', 'zone_2_x', 'zone_2_y','zone_3_x', 'zone_3_y', 'zone_4_x', 'zone_4_y',
           'zone_5_x', 'zone_5_y']
    surr_zones_ttc_pd = pd.DataFrame(surr_zones_ttc)
    surr_zones_ttc_pd.columns = cols
    return surr_zones_ttc_pd 

In [11]:
import pickle
def preprocess():
    pickle_name = 'pickle/Risk_DATA_' + str(TIME_VARIANTS[0]) + '.pkl'
    with open(pickle_name, "rb") as f:
        DATA = pickle.load(f)
        
    for i in range(len(DATA)):
        lane_change_id = DATA[i]['lane_change_id']
        risk_labels = DATA[i]['risk_labels']
        print(risk_labels)
        drv_data = DATA[i]['drv_data']
        surr_info = DATA[i]['surr_info']
        dynamic_drv = get_dynamic_data(drv_data)
        surr_zones = divide_surr_zone(surr_info)
        DATA[i]['dynamic_drv'] = dynamic_drv    
        DATA[i]['surr_zones'] = surr_zones

        if i % 100 == 0:
            print(i)
        
    pickle_name = 'pickle/risk_dynamic_data_' + str(TIME_VARIANTS[0]) + '.pkl'
    with open(pickle_name, "wb") as f:
        pickle.dump(DATA, f) #保存
        print('Data saved.')
preprocess()       

score_1                 3.00000
score_2                 1.00000
score_3                 1.00000
score_4                 1.00000
score_5                 1.00000
score_6                 1.00000
score_7                 2.00000
score_8                 3.00000
score_9                 2.00000
score_10                2.00000
normalized_risk        -0.30375
objective_risk_label    0.00000
S_P1                    0.00000
S_P2                    0.00000
S_P3                    0.00000
S_P4                    0.00000
S_P5                    0.00000
S_P6                    0.00000
S_P7                    0.00000
S_P8                    0.00000
S_P9                    0.00000
S_P10                   0.00000
Name: 0, dtype: float64
0
score_1                 2.00000
score_2                 1.00000
score_3                 1.00000
score_4                 4.00000
score_5                 2.00000
score_6                 1.00000
score_7                 2.00000
score_8                 2.00000
score_9       

In [12]:
def show_data():
    pickle_name = 'pickle/risk_dynamic_data_' + str(TIME_VARIANTS[0]) + '.pkl'
    with open(pickle_name, "rb") as f:
        DATA = pickle.load(f)
    
    for i in range(1):
        lane_change_id = DATA[i]['lane_change_id']
        risk_labels = DATA[i]['risk_labels']
        drv_data = DATA[i]['drv_data']
        surr_info = DATA[i]['surr_info']       
        dynamic_drv = DATA[i]['dynamic_drv']
        surr_zones = DATA[i]['surr_zones']
        
        
        print(DATA[i]['risk_labels'])
        print(DATA[i]['dynamic_drv'])
        print(DATA[i]['surr_zones'])

show_data()  
    

score_1                 3.00000
score_2                 1.00000
score_3                 1.00000
score_4                 1.00000
score_5                 1.00000
score_6                 1.00000
score_7                 2.00000
score_8                 3.00000
score_9                 2.00000
score_10                2.00000
normalized_risk        -0.30375
objective_risk_label    0.00000
S_P1                    0.00000
S_P2                    0.00000
S_P3                    0.00000
S_P4                    0.00000
S_P5                    0.00000
S_P6                    0.00000
S_P7                    0.00000
S_P8                    0.00000
S_P9                    0.00000
S_P10                   0.00000
Name: 0, dtype: float64
       brake   accel  velocity  steering_angle   a_lon   a_lat  \
6357  0.1470  6.2237   88.6604         -0.7118  0.0182  0.0364   
6358  0.1874  5.4060   88.6576         -2.2220  0.0165  0.0346   
6359  0.2362  4.5534   88.6754         -3.4473  0.0152  0.0338   
6360  0.

# 2. Driving style capturing
1. 10 drivers driving style
2. 10 subjective risk evaluators

In [38]:
DRIVER_IDS = ['m9600A', 'm9601A', 'm9602B','m9603A', 'm9604A', 'f9605A',
             'm9606A', 'f9608A', 'm9609A', 'm9611A']
EXPERT_IDS = ['m9600A', 'm9601A', 'm9602B','m9603A', 'm9604A']
NORMOAL_IDS = ['f9605A', 'm9606A', 'f9608A', 'm9609A', 'm9611A']
FEMALE_IDS = ['f9605A', 'f9608A']
MALE_IDS = ['m9600A', 'm9601A', 'm9602B','m9603A', 'm9604A', 'm9606A', 'm9609A', 'm9611A']

def get_data_each_driver():
    
    pickle_name = 'pickle/dynamic_data_' + str(TIME_VARIANTS[0]) + '.pkl'
    with open(pickle_name, "rb") as f:
        DATA = pickle.load(f)
    data_each_driver = []
    
    for i_driver in DRIVER_IDS:
        cnt = 0
        driver_data = dict(driver_id = i_driver, dynamic_drv = [], surr_zones = [] )
        for i in range(len(DATA)):
            risk_labels = DATA[i]['risk_labels']
            driver = risk_labels['driver']
            if i_driver == driver:
                if cnt == 0:   
                    driver_data['dynamic_drv'] = DATA[i]['dynamic_drv']
                    driver_data['surr_zones'] = DATA[i]['surr_zones']
                    cnt += 1
                else:

                    df1 = driver_data['dynamic_drv']
                    df2 = DATA[i]['dynamic_drv']
                    driver_data['dynamic_drv'] = pd.concat([df1, df2])
                    
                    df3 = driver_data['surr_zones']
                    df4 = DATA[i]['surr_zones']
                    driver_data['surr_zones'] = pd.concat([df3, df4])  
            else:
                pass
        data_each_driver.append(driver_data)
        #print(data_each_driver[0]['dynamic_drv'])
        
        pickle_name = 'pickle/driver_data_' + str(TIME_VARIANTS[0]) + '.pkl'
        with open(pickle_name, "wb") as f:
            pickle.dump(data_each_driver, f) #保存
        print('Data saved.')

get_data_each_driver()  

Data saved.
Data saved.
Data saved.
Data saved.
Data saved.
Data saved.
Data saved.
Data saved.
Data saved.
Data saved.


# How to define Personalized driver models

- **Personalized driver models** are learned from the data collected from the specific driver. For instance, the personalized model of driver \#1 is learned using the driving data collected from driver \#1.
- **Average driver models** are learned using the data collected from other nine drivers.

In [ ]:
# Driving style 
# PCA

def driver_model_PCA():
    pickle_name = 'pickle/driver_data_' + str(TIME_VARIANTS[0]) + '.pkl'
    with open(pickle_name, "rb") as f:
        DATA = pickle.load(f)
        
driver_model_PCA()       



#### 3. Simulation
- Lane change scenarios

#### 4. Personalized data driven control
1. Personalized MPC
2. Risk adaptive control
3. Data-driven control
  - Training phase
  - Inference phase


#### 5.Experimental result
- KL divergence
- MSE
- Interrupting time
- Subjective risk evaluation